<h1> Making an predictive model using Gradient Boosting Machine</h1>
In deze jupyter notebook file, worden de de decision tree gemaakt. 
Dit wordt gedaan om de onderzoeksvraag van mijn thesis te kunnen beantwoorden:
To what extent can support vector machine, randomforest tree, or Gradient Boosting Machine contributeto predicting the demand for the specialist youth caresegments in Amsterdam?
Ook is dit nodig voor het beantwoorden van mijn sub vraen:
•Are there neighborhood socio-demographic characteristics which are predictive of the use of youth caresegments?
•Which of the tested models has the highest f1 score in predicting the youth care segment use?


<h3>Hier onder worden eerst de benodigde librabry geimporteerd</h3>

In [1]:
import pandas as pd
import xgboost
import time
import numpy as np
import matplotlib.pylab as plt
from matplotlib import pyplot

from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from numpy import mean
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from xgboost import plot_importance


Om de sub onderzoeks vraag: Which of the tested models has the highest f1-score in predicting the youth care segment use? Waarom we voor deze score hebben gekozen, kan gelezen worden onder het kopje "model eveluation".

Ook maken we een aantal variabele hier aan om de code zo gestructuurd mogelijk te houden. Waarom deze nodig zijn, valt te lezen in het kopje "model making'

In [2]:
# define some variables
param_grid = {
        'max_depth': range(4,26,4),
        'scale_pos_weight' : [1, 25, 50, 75, 100],
        'colsample_bytree': np.arange(0.5,1.0,0.3),
}


cv_method = RepeatedStratifiedKFold(n_splits=2, 
                                    n_repeats=1, 
                                    random_state=42)


<h3> Laad de data in, die gemaakt is uit de andere jupyter notebook file</h3

In [3]:
df = pd.read_pickle("C:\\VERTROUWELIJK\\final_dataSet.pkl")

Split the data in indepentend variable an dependent variable. 

In [4]:
X = df.drop(['Voorziening'], axis=1)
X_encoded = pd.get_dummies(X, columns=['Geslacht'])
y = df['Voorziening'].copy()

encode string class values as integers, which is needed for Gradient Boosting

In [5]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

Make train and test set. Waarom dit nodig is, zie "making model" 

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_encoded, label_encoded_y)

<h3>Make the first gradient Boosting</h3>
and fit this to get the scores. Dit is nodig om alle onderzoeksvragen mee te bentwoorden.
Zie "model" in de thesis waarom we voro xgboost hebben gekozen en niet noor de standaard aangeleverde vanuit sklearn

In [7]:
# fit model no training data
model = xgboost.XGBClassifier()
start_time=time.time()
# model.fit(X_train, y_train)
print(model)
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_micro')
score = mean(scores)
print("f1_score: %.2f%%" % (score * 100.0))
std_scores = scores.std()
print("f1_score STD")
print(std_scores)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)
f1_score: 60.38%
f1_score STD
0.005676588725797207


Zoals gezegd, de data is erg imbalanced. Daarom maken we een decision tree with random undersampling

In [8]:
steps = [('under', RandomUnderSampler()), ('model', xgboost.XGBClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=42)
scores = cross_val_score(pipeline, X_encoded, label_encoded_y, scoring='f1_micro', cv=cv, n_jobs=-1, verbose = 10)
score = mean(scores)
print('F1 Score: %.3f' % score)
std_scores = scores.std()
print("f1_score STD")
print(std_scores)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


F1 Score: 0.249
f1_score STD
0.0057331714555510505


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   48.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   48.0s finished


<h3>Prune the model</h3>
Na het vergelijken van de twee modellen, heeft het model met de normale data set de hoogste F1 score, hier na gaan we het model nog prunen. Zie Making Model in de thesis

In [9]:
# make an gridSearch
start_time=time.time()
grid = GridSearchCV(xgboost.XGBClassifier(),
                   param_grid=param_grid,
                    scoring='f1_micro',
                   verbose=10,
                    cv= cv_method,
                   n_jobs=-1)
start_time=time.time()

grid.fit(X_train,y_train)
print(time.time()-start_time)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 2 folds for each of 60 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 20.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 27.3min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 32.6min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 39.3min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 50.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 63.4min
[Parallel(n_jobs=-1)]: Done 118 out of 120 | elapsed: 78.3min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 79.5min finished


[20:57:59] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


4816.003154754639


Getting the scores of the GridSearch

In [10]:
grid.cv_results_['std_test_score'][grid.best_index_]

0.002120629454342604

In [11]:
# get the best parameters of coming from the gridSearch.
grid.best_params_

{'colsample_bytree': 0.8, 'max_depth': 4, 'scale_pos_weight': 1}

In [12]:
# get the best F1 score, coming from the gridSearch.
grid.best_score_

0.5792789028356375

Om de onderzoeks vraag: Are there neighborhood socio-demographic characteristics which are predictive of the use of youth caresegments? is onderstaande code nodig. Echter voeren wij deze niet uit, omdat Decicion tree een hogere F1 score heeft. 

In [13]:
# Plot the top 10 most important features of the XGboost model
# xgboost.plot_importance(model, max_num_features=10)
# plt.show()